# 7. Classify companies for statistical tests 
-------------------
Group 3 , September 24, 2022
1. Gezhi Cheng, 
2. Haowei Lee, 
3. Ziyi Liu, 
4. VS Chaitanya Madduri

## Load dependencies

In [1]:
# Mount the program to the folder on Google Drive to get access to other files on cloud
from google.colab import drive
drive.mount('/content/drive')

# import python packages
import pandas as pd
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# define the path of the root folder and of the data
DIR_PATH = "/content/drive/MyDrive/Colab Notebooks/SPM Final Project/"  # take the current folder as root folder
RESTRUCTURING_DATA_PATH = "restructuring_data.csv"
LIMIT =3

COLUMNS_ORDER = ["Company_Key", "Year", 'compamy_name', "profitability", 'profitability_next_1',
       'profitability_next_2', 'profitability_next_3', 'change_next_1',
       'change_next_2', 'change_next_3', 'is_restructuring']

IS_RESTRUCTURING = "is_restructuring"
POSITIVE_CHANGE_1 = "positive_change_1"
POSITIVE_CHANGE_2 = "positive_change_2"
POSITIVE_CHANGE_3 = "positive_change_3"
CLASS = "class"

PREFIX_ONE = "A"
PREFIX_TWO = "B" 
PREFIX_THREE = "C" 

In [ ]:
df = pd.read_csv(DIR_PATH + RESTRUCTURING_DATA_PATH)  # read in the data with profitability and restructuring frequency
df = df.sort_values(by=['Company_Key', 'Year'])       # sort the records by company number and year
df = df[COLUMNS_ORDER]                                # sort the colum by given order
df = df.drop_duplicates(subset=['Company_Key', 'Year'], keep='last')
print(df.shape)
df.head(15)

In [3]:
# initiate the positive change columns, and set default value to zero
df[POSITIVE_CHANGE_1] = 0
df[POSITIVE_CHANGE_2] = 0
df[POSITIVE_CHANGE_3] = 0

# classify positive changes as value one, and negative ones as zero
df[POSITIVE_CHANGE_1] = df["change_next_1"].apply(lambda x: 1 if x > 0 else 0)
df[POSITIVE_CHANGE_2] = df["change_next_2"].apply(lambda x: 1 if x > 0 else 0)
df[POSITIVE_CHANGE_3] = df["change_next_3"].apply(lambda x: 1 if x > 0 else 0)

df.head(LIMIT)

,Company_Key,Year,compamy_name,profitability,profitability_next_1,profitability_next_2,profitability_next_3,change_next_1,change_next_2,change_next_3,is_restructuring,positive_change_1,positive_change_2,positive_change_3
204,4904,2011,AEP Texas Central Transition Funding II LLC,0.0322,0.0255,0.0272,0.0221,-20.91,-15.54,-31.33,0,0,0,0
48,4904,2012,AMERICAN ELECTRIC POWER CO INC,0.0255,0.0272,0.0221,0.0420,6.79,-13.17,65.03,1,1,0,1
241,4904,2013,OHIO POWER CO,0.0272,0.0221,0.0420,0.0134,-18.69,54.54,-50.69,0,0,1,0


In [4]:
df_y1 = df.copy()
df_y2 = df.copy()
df_y3 = df.copy()

df_y1[CLASS] = 0
df_y2[CLASS] = 0
df_y3[CLASS] = 0

In [5]:


def get_class(df, year_gap, restructure_col, profit_col):

    if year_gap == 1:
        pre = PREFIX_ONE
    elif year_gap == 2:
       pre = PREFIX_TWO
    elif year_gap == 3:
       pre = PREFIX_THREE

    filter_1 = (df[restructure_col] == 1) & (df[profit_col] == 0)
    filter_2 = (df[restructure_col] == 1) & (df[profit_col] == 1)
    filter_3 = (df[restructure_col] == 0) & (df[profit_col] == 0)
    filter_4 = (df[restructure_col] == 0) & (df[profit_col] == 1)


    df[CLASS][filter_1] = f"{pre}1"
    df[CLASS][filter_2] = f"{pre}2"
    df[CLASS][filter_3] = f"{pre}3"
    df[CLASS][filter_4] = f"{pre}4"

    return df

In [6]:
import warnings
warnings.filterwarnings("ignore")

dfs = [(df_y1, POSITIVE_CHANGE_1),
       (df_y2, POSITIVE_CHANGE_2),
       (df_y3, POSITIVE_CHANGE_3)]

for i in range(len(dfs)):
    df, change = dfs[i][0], dfs[i][1]
    df = df.pipe(get_class, i+1, IS_RESTRUCTURING, change)
    print(df[CLASS].value_counts())

A3    173
A4    133
A1     63
A2     60
Name: class, dtype: int64
B3    164
B4    142
B2     69
B1     54
Name: class, dtype: int64
C3    173
C4    133
C2     72
C1     51
Name: class, dtype: int64
